In [1]:
import os
import numpy as np
import pandas as pd
import gc
import time
# from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import warnings
import numpy as np
import re
PATH = './'

total = pd.read_csv('./TEG 7-2.csv')
total.columns

Index(['ID', '病案号', '病人姓名', '年龄', 'tel', '主诊断（1oc2.EC 3.CC4.sarcoma）',
       '分化（1高分化2中分化3低分化4无分化）', '病理', '分期', '手术', '手术方式(1.开腹 2lap)', '病人姓名.1',
       '手术日期', '手术时长（min）', '术中输血1.有0无', '术中输血量', '术中出血（ml）', '病人姓名.2', '冠心病',
       '高血压', '高血脂', '血栓病史', '糖尿病', 'R', 'k', 'angle', 'MA', 'CI', 'G', 'EPL',
       'PT', 'INR', 'APTT', 'Fbg', 'D-D', 'T-T', '病人姓名.3', 'R.1', 'k.1',
       'angle.1', 'MA.1', 'CI.1', 'G.1', 'EPL.1', 'PT.1', 'INR.1', 'APTT.1',
       'FbG', 'D-D.1', 'T-T.1', 'PC', '术前超声有无血栓(1有0无)', '术后超声', 'VTE', 'VTE时间',
       'PE', 'PE时间', 'DVT', 'DVT时间', '处理（1.2.3.）', '转归（1.*2.*3.死亡）', '其他'],
      dtype='object')

In [2]:
use = ['年龄', '主诊断（1oc2.EC 3.CC4.sarcoma）', '分化（1高分化2中分化3低分化4无分化）', '手术', '手术时长（min）',
       '术中输血1.有0无', '术中出血（ml）', '冠心病', '高血压', '高血脂', '血栓病史', '糖尿病', 'R', 'k', 'angle', 'MA',
       'CI', 'G', 'EPL', 'PT', 'INR', 'APTT', 'Fbg', 'D-D', 'T-T', 'R.1', 'k.1', 'angle.1', 'MA.1', 'CI.1', 'G.1',
       'EPL.1', 'PT.1', 'INR.1', 'APTT.1', 'FbG', 'D-D.1', 'T-T.1', '术前超声有无血栓(1有0无)', '术后超声']

df = total[use].rename(columns={'术后超声':'l'})

df['R'].replace(' ', np.nan, inplace=True)
df['R'] = df['R'].apply(lambda x: float(x))
df['术前超声有无血栓(1有0无)'] = df['术前超声有无血栓(1有0无)'].fillna(3).astype('int8')
def f(x):
    x = re.sub(' ', '3', x)
    return int(x)
df['l'] = df['l'].fillna('3').apply(f)
df.shape

年龄                             True
主诊断（1oc2.EC 3.CC4.sarcoma）     True
分化（1高分化2中分化3低分化4无分化）           True
手术                             True
手术时长（min）                      True
术中输血1.有0无                      True
术中出血（ml）                       True
冠心病                            True
高血压                            True
高血脂                            True
血栓病史                           True
糖尿病                            True
R                              True
k                              True
angle                          True
MA                             True
CI                             True
G                              True
EPL                            True
PT                             True
INR                            True
APTT                           True
Fbg                            True
D-D                            True
T-T                            True
R.1                            True
k.1                            True
angle.1                     

In [3]:
df['主诊断（1oc2.EC 3.CC4.sarcoma）'].unique()

array(['1', '2', '3', nan, '外阴癌', '原发灶不明'], dtype=object)

In [4]:
cat_feats = ['主诊断（1oc2.EC 3.CC4.sarcoma）']
for col in cat_feats:
    df[col],_ = pd.factorize(df[col])
cat_feats = ['主诊断（1oc2.EC 3.CC4.sarcoma）', '分化（1高分化2中分化3低分化4无分化）', '手术',
             '术中输血1.有0无', '冠心病', '高血压', '高血脂', '血栓病史', '糖尿病', '术前超声有无血栓(1有0无)']
num_feats = list(set(df.columns) - set(cat_feats + ['l']))
len(num_feats)

['D-D',
 'angle.1',
 'INR.1',
 '手术时长（min）',
 'D-D.1',
 '年龄',
 'EPL',
 'T-T',
 'R.1',
 'R',
 'INR',
 'FbG',
 'APTT.1',
 'PT.1',
 'CI',
 'MA.1',
 'MA',
 'Fbg',
 'PT',
 'G',
 'k.1',
 'APTT',
 '术中出血（ml）',
 'angle',
 'CI.1',
 'G.1',
 'T-T.1',
 'EPL.1',
 'k']

In [7]:
def ratio_name(col1, col2):
    return col1 + '_divide_' + col2
def ratio(df, col1, col2):
    return df[col1] / (df[col2] + 1e-3)
def times_name(col1, col2):
    return col1 + '_times_' + col2
def times(df, col1, col2):
    return df[col1] * df[col2]
def plus_name(col1, col2):
    return col1 + '_add_' + col2
def plus(df, col1, col2):
    return df[col1] + df[col2]
def minus_name(col1, col2):
    return col1 + '_minus_' + col2
def minus(df, col1, col2):
    return df[col1] - df[col2]


total_num_feats = num_feats.copy()
for x in num_feats:
    for y in num_feats:
        if ratio_name(y, x) not in df.columns and x != y:
            total_num_feats.append(ratio_name(x, y))
            df[ratio_name(x, y)] = ratio(df, x, y)
            
            total_num_feats.append(times_name(x, y))
            df[times_name(x, y)] = times(df, x, y)
            
            total_num_feats.append(plus_name(x, y))
            df[plus_name(x, y)] = plus(df, x, y)
            
            total_num_feats.append(minus_name(x, y))
            df[minus_name(x, y)] = minus(df, x, y)
len(total_num_feats)

1653

In [56]:
from sklearn.cross_validation import train_test_split
pos_idx = df[df['l'] == 1].index.values.tolist()
neg_idx = df[df['l'] == 0].index.values.tolist()
idx = pos_idx + neg_idx
test_idx = np.random.choice(pos_idx, 2, replace=False).tolist() + np.random.choice(neg_idx, 22, replace=False).tolist()
train_idx = list(set(idx) - set(test_idx))
train = df.loc[train_idx]
test = df.loc[test_idx]
set(test_idx) & set(pos_idx)

{25, 57}

In [ ]:
lgbm_train = lgbm.Dataset(data=train,
                          label=target,
                          categorical_feature=cat_feats,
                          free_raw_data=False)
lgbm_params = {
    'task': 'train',
    'boosting_type': 'gbdt',
    'objective': 'binary',
    'metric': 'auc',
    'learning_rate': 0.02,
    'num_iteration': 4000,
    'num_threads': 8,

    'num_leaves': int(round(44.368535336628419)),
    'feature_fraction': 0.15,
    'bagging_fraction': 0.94901525271474951,
    'max_depth': int(round(8.0430115561596267)),
    'lambda_l1': 0.30680079516647751,
    'lambda_l2': 0.079128660903201031,
    'min_split_gain': 0.054005067457890979,
    'min_child_weight': 15.172643147364937}
cv_results = lgbm.cv(train_set=lgbm_train,
                     params=lgbm_params,
                     nfold=5,
                     early_stopping_rounds=150,
                     verbose_eval=100,
                     metrics=['auc'])

optimum_boost_rounds = np.argmax(cv_results['auc-mean'])
print('Optimum boost rounds = {}'.format(optimum_boost_rounds))
print('Best CV result = {}'.format(np.max(cv_results['auc-mean'])))

In [ ]:
clf = lgbm.train(train_set=lgbm_train,
                 params=lgbm_params,
                 num_boost_round=optimum_boost_rounds,
                verbose_eval=100)

""" Predict on test set and create submission """
y_pred = clf.predict(test)
out_df = pd.DataFrame({'SK_ID_CURR': test_df['SK_ID_CURR'], 'TARGET': y_pred})

In [ ]:
x = [x for x in zip(clf.feature_name(), clf.feature_importance('gain'))]
x.sort(key=lambda x: x[1], reverse=True)
x = [(i, j) for i, j in enumerate(x)]
x

In [ ]:
x = [x for x in zip(clf.feature_name(), clf.feature_importance('split'))]
x.sort(key=lambda x: x[1], reverse=True)
x = [(i, j) for i, j in enumerate(x)]
x